<a href="https://colab.research.google.com/github/sarkar2527/DSAP/blob/main/Maleria_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 20:13:35 2021

@author: Hi
"""

import pandas as pd

import os
my_data_dir= "D:\\CARRER\\My_Course\\Data Science Classes\\4 Module\\2 DEEP LEARNING\\5 Tensorflow2-Keras-DeepLeanring\\04-CNNs\\cell_images"
os.listdir(my_data_dir)

test_path = my_data_dir+"\\test\\"
train_path = my_data_dir+"\\train\\" 


#====================================================
# To verify any image as a test
para_img = train_path+"\\parasitized\\"+"\\C33P1thinF_IMG_20150619_114756a_cell_179.png"

from matplotlib.image import imread
para_cell_img = imread(para_img)
para_cell_img.shape

import matplotlib.pyplot as plt
plt.imshow(para_cell_img)
#====================================================


len(os.listdir(train_path+"\\parasitized\\"))
len(os.listdir(train_path+"\\uninfected\\"))
len(os.listdir(test_path+"\\parasitized\\"))
len(os.listdir(test_path+"\\uninfected\\"))


#====================================================
# image pre-processing work

from tensorflow.keras.preprocessing.image import ImageDataGenerator
help(ImageDataGenerator)
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.10,
                               height_shift_range=0.10,
                               rescale=1/255,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode="nearest")


image_shape = (130,130,3)
train_image_gen = image_gen.flow_from_directory(train_path,
                              target_size=image_shape[:2],
                              color_mode='rgb',
                              batch_size=500,
                              class_mode='binary')

test_image_gen = image_gen.flow_from_directory(test_path,
                              target_size=image_shape[:2],
                              color_mode='rgb',
                              batch_size=500,
                              class_mode='binary')



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D,Flatten


model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(14,14), input_shape=image_shape))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation="relu")) # inputlayer
model.add(Dense(1, activation="sigmoid")) # output

# compiler
model.compile(optimizer= "adam", loss="binary_crossentropy",metrics=["accuracy"])

# Training the model
model.fit(train_image_gen, validation_data=test_image_gen, epochs=2)

import pandas as pd
losses = pd.DataFrame(model.history.history)
losses
losses[['loss','val_loss']].plot()
losses[['accuracy','val_accuracy']].plot()

y_pred = model.predict_generator(test_image_gen)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_image_gen.classes,y_pred)
cm